In [1]:
from pynwb.spec import NWBNamespaceBuilder, NWBGroupSpec, NWBAttributeSpec

ns_path = "test.namespace.yaml"
ext_source = "test.extensions.yaml"

ns_builder = NWBNamespaceBuilder('Test extension', "test", version='0.1.0')

ns_builder.include_type('NWBContainer', namespace='core')

ext = NWBGroupSpec(
        neurodata_type_def='SpatialLightModulator',
        neurodata_type_inc='NWBContainer',
        doc=('slm'),
        attributes=[
            NWBAttributeSpec(
                # name='size',
                doc='vertices for surface, points in 3D space',
                shape=(2, ),
                name='size', dtype='float'
            )
        ])

ns_builder.add_spec(ext_source, ext)

from pynwb.spec import NWBDatasetSpec, NWBDtypeSpec, RefSpec

ext = NWBGroupSpec(
        neurodata_type_def='PhotostimulationDevice',
        neurodata_type_inc='NWBContainer',
        doc=('photostimDevice'),
        attributes=[
            NWBAttributeSpec(
                # name='size',
                name='slm',
                doc='slm',
                dtype=RefSpec('GroupSpec', 'object')
            )
        ])

ns_builder.add_spec(ext_source, ext)
ns_builder.export(ns_path)


In [2]:
from pynwb import register_class, load_namespaces
from pynwb.ecephys import ElectricalSeries
from hdmf.utils import docval, get_docval, popargs
from pynwb.core import NWBContainer
from collections.abc import Iterable


ns_path = "test.namespace.yaml"
load_namespaces(ns_path)


@register_class('SpatialLightModulator', 'test')
class SpatialLightModulator(NWBContainer):

    __nwbfields__ = ('size',)

    @docval(*get_docval(NWBContainer.__init__) + (
        {'name': 'size', 'type': Iterable, 'doc': 'the tetrode id'},))
    def __init__(self, **kwargs):
        size = popargs('size', kwargs)
        super().__init__(**kwargs)
        self.size = size

@register_class('PhotostimulationDevice', 'test')
class PhotostimulationDevice(NWBContainer):

    __nwbfields__ = ('slm',)

    @docval(*get_docval(NWBContainer.__init__) + (
        {'name': 'slm', 'type': SpatialLightModulator, 'doc': 'the tetrode id'},))
    def __init__(self, **kwargs):
        slm = popargs('slm', kwargs)
        super().__init__(**kwargs)
        self.slm = slm

In [4]:

from datetime import datetime
from dateutil.tz import tzlocal

import numpy as np
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.image import ImageSeries
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, \
    Fluorescence, CorrectedImageStack, MotionCorrection, RoiResponseSeries

import matplotlib.pyplot as plt

import importlib
import ndx_photostim
importlib.reload(ndx_photostim)

nwbfile = NWBFile(
    'my first synthetic recording',
    'EXAMPLE_ID',
    datetime.now(tzlocal()),
)

import numpy as np
slm = SpatialLightModulator(
    name="OpticalChannel",
    size = [1, 2],#[32, 10],
)

dev = PhotostimulationDevice(name="device", slm=slm)
print(dev)

print(nwbfile)

nwbfile.add_device(slm)

print(nwbfile)

device __main__.PhotostimulationDevice at 0x140717368021248
Fields:
  slm: OpticalChannel __main__.SpatialLightModulator at 0x140716819804272
Fields:
  size: [1 2]


root pynwb.file.NWBFile at 0x140716819716896
Fields:
  file_create_date: [datetime.datetime(2022, 9, 20, 12, 48, 9, 906028, tzinfo=tzlocal())]
  identifier: EXAMPLE_ID
  session_description: my first synthetic recording
  session_start_time: 2022-09-20 12:48:09.905726-04:00
  timestamps_reference_time: 2022-09-20 12:48:09.905726-04:00



TypeError: MultiContainerInterface.__make_add.<locals>._func: incorrect type for 'devices' (got 'SpatialLightModulator', expected 'list, tuple, dict or Device')